In [1]:
# %conda install -c conda-forge ta-lib -q
# %pip install -r requirements.txt -q

# import boto3
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy
import time

from data import * 
from feature_engineering import FeaturesEngineering
from models import Models
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-02-25 20:12:17.916683: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-25 20:12:18.038685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740485538.082531    4511 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740485538.096835    4511 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 20:12:18.221800: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
pypickle_overwrite = True

models = Models(use_xgboost_gpu=True)

In [3]:
dir_project = os.getcwd()
dir_models = os.path.join(dir_project, 'models')
dir_models_base = os.path.join(dir_project, 'models', 'base')
dir_models_scaler = os.path.join(dir_project, 'models', 'scaler')
dir_models_best = os.path.join(dir_project, 'models', 'best')

print(f"""
Project directory: {dir_project}
Models directory: {dir_models}
Base models directory: {dir_models_base}
Scalers directory: {dir_models_scaler}
Best models directory: {dir_models_best}
""")


Project directory: /home/alfred/projects/mscfe-masters-project/src
Models directory: /home/alfred/projects/mscfe-masters-project/src/models
Base models directory: /home/alfred/projects/mscfe-masters-project/src/models/base
Scalers directory: /home/alfred/projects/mscfe-masters-project/src/models/scaler
Best models directory: /home/alfred/projects/mscfe-masters-project/src/models/best



# Read raw data

In [4]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [5]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full.csv'
df_full = pd.read_csv('df_full.csv').drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
277941,2024-12-11 17:00:00,3817.8100,3821.830,3812.87,3821.8300,ETHUSDT
155261,2024-10-30 02:40:00,2685.5100,2691.040,2684.77,2689.7500,ETHUSDT
4207,2024-10-17 18:50:00,164.6800,164.735,164.53,164.5900,GOOG
331752,2024-12-30 09:25:00,188.2300,188.580,187.97,188.4500,SOLUSDT
30856,2024-11-19 13:20:00,142.8099,142.980,142.55,142.8901,NVDA


## Placeholders

In [6]:
dict_data = {}
dict_cv_features = {}

# Features Engineering

## Create and normalizing features

In [7]:
# Initializing the FeaturesEngineering obj
feat = FeaturesEngineering()

# Default config
feat.default_config

{'ema': [8, 21, 55],
 'macd': {'fast': 12, 'slow': 26, 'signal': 9},
 'rsi': [14],
 'bb': {'timeperiod': 20, 'nbdevup': 2, 'nbdevdn': 2},
 'atr': {'timeperiod': 14},
 'stoch': {'fastk_period': 14, 'slowk_period': 3, 'slowd_period': 3},
 'cci': {'timeperiod': 20},
 'willr': {'timeperiod': 14}}

1. Create features from pairs data
2. Load saved scaler
3. Perform normalization / standardization

In [8]:
scaler_type = "StandardScaler"
dict_data['df_feat'] = {}
dict_data['df_norm'] = {}
dict_data['scaler'] = {}
i = 1

for p in pairs:

    print(f"({i}) Pairs: {p}")
    tickerX, tickerY = p.split(' ')
    
    # 1. Create features from pairs data
    df_feat = feat.create_features(
        tickerX, 
        tickerY,
        df_full[df_full['Symbol']==tickerX],
        df_full[df_full['Symbol']==tickerY],
        config=None, # using default config in the FeaturesEngineering
        dropna=True
    )

    # 2. Load saved scaler
    scaler_name = f"{scaler_type.lower()}_{p.replace(' ', '_')}.pkl"
    dir_scaler = os.path.join(dir_models_scaler, scaler_name)
    scaler = pypickle.load(dir_scaler)

    # 3. Perform normalization / standardization
    data_scaled = scaler.transform(df_feat)
    df_norm = pd.DataFrame(
        data_scaled, index=df_feat.index, columns=df_feat.columns
    )
    df_norm = df_norm.rename(columns={'Spread':'NormalizedSpread'})
    
    dict_data['df_feat'][p] = df_feat
    dict_data['df_norm'][p] = df_norm
    dict_data['scaler'][p] = scaler

    i+=1
    print('')

(1) Pairs: META DAXEX_ETF
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_ETHUSDT.pkl]

(6) Pairs: EQQQ_ETF BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_EQQQ_ETF_BNBUSDT.pkl]

(7) Pairs: BNBUSDT ADAUSDT
[pypickle] Pickle file loaded: [/ho

## Read LassoCV selected features

In [9]:
lasso_cv_name = f"lasso_cv_features.pkl"
dir_lasso = os.path.join(dir_models, lasso_cv_name)
dict_cv_features = pypickle.load(dir_lasso)

for key in dict_cv_features:
    print(f"{key}: {dict_cv_features[key]}")

[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/lasso_cv_features.pkl]
META DAXEX_ETF: ['META_EMA8', 'META_EMA55', 'META_RSI14', 'META_ATR14', 'META_StochD3', 'META_CCI20', 'META_WILLR14', 'DAXEX_ETF_EMA8', 'DAXEX_ETF_EMA21', 'DAXEX_ETF_MACD', 'DAXEX_ETF_BBupper', 'DAXEX_ETF_BBmiddle', 'DAXEX_ETF_BBlower', 'DAXEX_ETF_StochD3']
AMZN BNBUSDT: ['AMZN_EMA8', 'AMZN_EMA21', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_WILLR14', 'BNBUSDT_EMA8', 'BNBUSDT_EMA55', 'BNBUSDT_MACD', 'BNBUSDT_RSI14', 'BNBUSDT_ATR14', 'BNBUSDT_StochD3', 'BNBUSDT_CCI20', 'BNBUSDT_WILLR14']
AMZN TRXUSDT: ['AMZN_EMA8', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_CCI20', 'AMZN_WILLR14', 'TRXUSDT_MACD']
SPY_ETF AVAXUSDT: ['SPY_ETF_EMA8', 'SPY_ETF_EMA55', 'SPY_ETF_MACD', 'SPY_ETF_RSI14', 'SPY_ETF_BBupper', 'SPY_ETF_BBmiddle', 'SPY_ETF_BBlower', 'SPY_ETF_ATR14', 'SPY_ETF_StochK14', 'SPY_ETF_StochD3', 'SPY_ETF_CCI20', 'SPY_ETF_WILLR14', 'AVAXUSDT_E

# Hyperparameters Tunning

## Transformer Regression

In [10]:
dir_best = os.path.join(dir_models_best, 'transformer')
param_grid = {
    'num_heads': [2, 4, 6],
    'lookback': [5, 10, 15],  # Number of steps to lookback
    'dropout_rate': [0.25],  # Dropout rates
    'batch_size': [32, 64], # Number of data per batch
    'ff_dim': [64, 128],
}

verbose = True
tuned_params = []
i = 1

for p in pairs:
    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    
    # 2. Create X, y based on LassoCV
    lasso_features_cols = dict_cv_features[p]
    target_col = df_norm.columns.tolist()[-1]
    df_lasso = pd.concat([df_norm[lasso_features_cols], df_norm[target_col]], axis=1)

    # 3. Run Hyperparameters tunning
    start_time = time.perf_counter()
    best_model, best_params, best_mse, best_df_test = models.hyperparameter_tuning(
        df=df_lasso, p=p, model_func=models.transformer_regression, param_grid=param_grid, verbose=verbose)
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    stats = {
        'pair':p, 
        'best_mse': best_mse,
        'data_points': len(df_lasso),
        'features': len(df_lasso.columns),
    }
    stats.update(best_params)
    tuned_params.append(stats)

    print(f"Best MSE: {best_mse:.6f}")
    print(f"num_heads: {best_params['num_heads']}, lookback: {best_params['lookback']}, ff_dim: {best_params['ff_dim']}, batch_size: {best_params['batch_size']}")
    print(f"Time taken: {elapsed_time:.6f} seconds")

    # 4. Save best model future use like testing and strategy
    model_name = f"best_lstm_{p.replace(' ', '_')}.pkl"
    dir_model_name = os.path.join(dir_best, model_name)
    pypickle.save(dir_model_name, best_model, overwrite=pypickle_overwrite)

    i+=1
    print('')

dir_param = os.path.join(dir_best, 'best_model_results.csv')
df_transformer = pd.DataFrame(tuned_params).set_index('pair')
df_transformer.to_csv(dir_param)
df_transformer(df_lstm)

(1) Pairs: META DAXEX_ETF
Testing params: {'num_heads': 2, 'lookback': 5, 'dropout_rate': 0.25, 'batch_size': 32, 'ff_dim': 64}


I0000 00:00:1740485574.367456    4511 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21829 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1740485576.774508    4682 service.cc:148] XLA service 0x70a8c8026280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740485576.774625    4682 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-02-25 20:12:56.821484: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740485577.040057    4682 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1740485579.794747    4682 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step
Transformer MSE: 0.0381280395328133
Testing params: {'num_heads': 2, 'lookback': 5, 'dropout_rate': 0.25, 'batch_size': 32, 'ff_dim': 128}
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step
Transformer MSE: 0.03304423892994814
Testing params: {'num_heads': 2, 'lookback': 5, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 64}
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step 
Transformer MSE: 0.04883287445130337
Testing params: {'num_heads': 2, 'lookback': 5, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 128}
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step 
Transformer MSE: 0.030140838231595187
Testing params: {'num_heads': 2, 'lookback': 10, 'dropout_rate': 0.25, 'batch_size': 32, 'ff_dim': 64}
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step
Transformer MSE: 0.03881038214306244
Testing params: {'num_heads': 2, 'lookback': 10, 'dropout_rate': 0.25, 'batch_size': 32, 'ff_dim': 128}
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step
Transformer MSE: 0.03484816042113051
Testing params: {'num_heads':

2025-02-25 20:14:10.188447: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 8 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_select_fusion', 8 bytes spill stores, 4 bytes spill loads



8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step 
Transformer MSE: 0.03813447079203745
Testing params: {'num_heads': 2, 'lookback': 10, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 128}


...

2025-02-25 22:59:25.739711: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_select_fusion', 56 bytes spill stores, 52 bytes spill loads



194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step    
Transformer MSE: 0.005272014459521054
Testing params: {'num_heads': 6, 'lookback': 15, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 128}


2025-02-25 23:00:00.456915: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_select_fusion', 56 bytes spill stores, 44 bytes spill loads



194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step    
Transformer MSE: 0.006890615004981486
Best parameters: {'num_heads': 6, 'lookback': 10, 'dropout_rate': 0.25, 'batch_size': 32, 'ff_dim': 64}, Best MSE: 0.0018562497744552518
Best MSE: 0.001856
num_heads: 6, lookback: 10, ff_dim: 64, batch_size: 32
Time taken: 1577.863419 seconds
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/transformer/best_lstm_BNBUSDT_XRPUSDT.pkl]



NameError: name 'df_lstm' is not defined

In [11]:
display(df_transformer)

,best_mse,data_points,features,num_heads,lookback,dropout_rate,batch_size,ff_dim
pair,,,,,,,,
META DAXEX_ETF,0.030082,1145,15,6,5,0.25,32,128
AMZN BNBUSDT,0.011686,4991,16,2,15,0.25,64,128
AMZN TRXUSDT,0.002809,4991,9,2,10,0.25,32,128
SPY_ETF AVAXUSDT,0.036903,4997,21,4,5,0.25,64,64
SPY_ETF ETHUSDT,0.006406,4997,16,2,5,0.25,64,128
EQQQ_ETF BNBUSDT,0.023795,3217,13,4,5,0.25,32,128
BNBUSDT ADAUSDT,0.001136,31051,9,4,15,0.25,32,64
BNBUSDT BTCUSDT,0.002063,31051,9,2,5,0.25,32,64
BNBUSDT TRXUSDT,0.001045,31051,12,6,15,0.25,32,64
